## 【問題1】K-meansクラスの作成

## K-meansモデルをスクラッチ開発する

公式ドキュメントより、必要な引数やパラメータは下記のとおりです。

### sklearn.cluster.KMeans
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

### 引数の把握


```python
KMeans(n_clusters=8, init=’k-means++’, n_init=10, max_iter=300, tol=0.0001, precompute_distances=’auto’, verbose=0, random_state=None, copy_x=True, n_jobs=None, algorithm=’auto’)
```

**zn_clusters : int, optional, default: 8**
* 形成するクラスターの数と生成する重心の数。

**init : {‘k-means++’, ‘random’ or an ndarray}**
* 初期化メソッド。デフォルトは 'k-means ++'。
* 'k-means ++'：収束をスピードアップするためのスマートな方法で、k-meanクラスタリング用の初期クラスタ中心を選択する。
* 'random'：初期重心のデータからランダムにk個の観測値（行）を選択する。
* 引数にndarrayが渡された場合、それは（n_clusters、n_features）にする必要がある。その上で最初の中心点をセットされる。

**n_init : int, default: 10**
* k-meansアルゴリズムが異なる重心で実行される回数。最終結果は、慣性に関してn_init連続実行の最良の出力になる。

**max_iter : int, default: 300**
* 1回の実行に対するk-meansアルゴリズムの最大反復回数。

**tol : float, default: 1e-4**
* 収束を宣言する慣性に関する相対許容誤差。

**precompute_distances : {‘auto’, True, False}**
* 距離を事前計算するか否か（高速ですが、より多くのメモリを使用する）。
* 'auto'：n_samples * n_clusters> 1200万の場合、距離を事前計算はしない。これは倍精度を使用すると、ジョブあたり約100MBのオーバーヘッドに相当するため。
* True：常に距離を事前計算する
* False：距離を事前計算しない

**verbose : int, default 0**
* 冗長モード

**random_state : int, RandomState instance or None (default)**
* 重心初期化のための乱数生成を決定する。ランダム性を決定論的にするためにintを使用する必要がある。

**copy_x : boolean, optional**
* 距離を事前計算するときは、最初にデータをセンタリングする方が数値的に正確。 copy_xがTrue（既定値）の場合、元のデータは変更されず、Xは確実にCに連続する。 Falseの場合、元のデータは変更され、関数が戻る前に元に戻され、データの平均値を減算してから加算することによって小さな数値の差が生じる可能性がある。

**n_jobs : int or None, optional (default=None)**
* 計算に使用するジョブの数。これは、n_initの各実行を並列に計算することによって機能する。
* joblib.parallel_backendコンテキストでない限り、Noneは1を意味する。 -1はすべてのプロセッサを使用することを意味する。

**algorithm : “auto”, “full” or “elkan”, default=”auto”**
* 使用するK平均アルゴリズム古典的なEMスタイルのアルゴリズムは「完全」。
* "elkan"バリエーションは三角不等式を使用することでより効率的になるが、スパースデータをサポートできない。
* “ auto”は、密データの場合は“ elkan”を選択し、疎データの場合は“ full”を自動選択する。

### メソッドの把握 fit
#### fit https://github.com/scikit-learn/scikit-learn/blob/7389dba/sklearn/cluster/k_means_.py#L943

```python
    def fit(self, X, y=None, sample_weight=None):
        """k平均クラスタリングを計算する
        Parameters
        ----------
        X : 配列型またはスパース行列、shape =（n_samples、n_features）
            クラスタリングするインスタンスをトレーニングします。データは
            Cの順序に変換されます。これによりメモリが発生します。
            与えられたデータがCに隣接していない場合はコピーします。
            
        y : （クラスタリングの特性上）無視します。
            使用されていません。規約によるAPIの一貫性のためにここに存在します。
            
        sample_weight：配列風、shape（n_samples、）、オプション
            Xの各観測値に対する重み。Noneの場合、すべての観測値
            等しい重みが割り当てられている（デフォルト：なし）
        """
        
        # シード（初期）データをnp.random.RandomStateインスタンスとして生成する
        random_state = check_random_state(self.random_state)

        # cluster_centers_：クラスターの中心座標を求める 
        # labels_：各点のラベル（compute_labelsがTrueに設定されている場合）を求める 
        #  inertia_：選択したものに関連する慣性基準の値パーティション（compute_labelsがTrueに設定されている場合）
        # n_iter_：反復数
        self.cluster_centers_, self.labels_, self.inertia_, self.n_iter_ = \
            k_means(
                X, n_clusters=self.n_clusters, sample_weight=sample_weight,
                init=self.init, n_init=self.n_init,
                max_iter=self.max_iter, verbose=self.verbose,
                precompute_distances=self.precompute_distances,
                tol=self.tol, random_state=random_state, copy_x=self.copy_x,
                n_jobs=self.n_jobs, algorithm=self.algorithm,
                return_n_iter=True)
        return self
```

### メソッドの把握 k_means

```python
def k_means(X, n_clusters, sample_weight=None, init='k-means++',
            precompute_distances='auto', n_init=10, max_iter=300,
            verbose=False, tol=1e-4, random_state=None, copy_x=True,
            n_jobs=None, algorithm="auto", return_n_iter=False):
    """K-means clustering algorithm.

    Parameters
    ----------
    X : 配列またはスパース行列、形状（n_samples、n_features）クラスター化するための観測データは
        Cの順序に変換されます。これにより、メモリコピーが発生します。与えられたデータがCに連続していない場合
        
    n_clusters : int
        形成するクラスターの数と数、生成する重心
        
    sample_weight : 配列風、形状（n_samples、）、オプション
        Xの各観測値に対する重み。Noneの場合、すべての観測値
        等しい重みが割り当てられている（デフォルト：なし）
        
    init :{'k-means ++'、 'random'、ndarray、または呼び出し可能}、オプション
        初期化メソッド。デフォルトは 'k-means ++'です。
        'k-means ++'：k-meanの初期クラスタ中心を選択
        賢い方法でクラスタリングして収束をスピードアップします。セクションを参照
        詳細はk_initのメモ。
        'random'：のデータからランダムにk個の観測値（行）を選択
        初期重心
        ndarrayが渡されるならば、それは形（n_clusters、n_features）であるべきです
        そして初期の中心を与える。
        呼び出し可能オブジェクトが渡されると、引数X、k、および
        ランダムな状態で初期化を返します。
        
    precompute_distances :{'auto'、True、False}
        距離を事前計算します（高速ですが、より多くのメモリを使用します）。
        'auto'：n_samples * n_clusters> 12の場合、距離を事前計算しません
        百万。これは、使用するジョブあたり約100MBのオーバーヘッドに相当します。
        倍精度。
        True：常に距離を事前計算します
        False：距離を事前計算しない
        
    n_init : int, optional, default: 10
        k-meansアルゴリズムが異なる値で実行される回数
        重心の種最終結果はの最良の出力になります
        n_initは連続して慣性の観点から実行されます。

    max_iter : int, optional, default 300
        実行するk-meansアルゴリズムの最大反復回数。
        詳細：ブール値、オプション：冗長モード
        
    tol : float, optional
        収束を宣言する前の結果の相対的な増分。
    random_state：int、RandomStateインスタンス、またはNone（デフォルト）
        重心初期化のための乱数生成を決定します。つかいます
        ランダム性を決定論的にするためのint値。
        ：term： `用語集<random_state>`を参照してください。
        
    copy_x : boolean, optional
        距離を事前計算するときは、中心合わせの方が数値的に正確です。
        最初にデータ。 copy_xがTrue（デフォルト）の場合、元のデータは
        変更されず、XがCに連続していることを確認します。 Falseの場合、元のデータ
        変更され、関数が戻る前に元に戻されますが、小さい
        数値の違いは、減算してから加算することで導入できます。
        データが意味する、この場合それはまたデータがであることを保証しないでしょう
        重大な減速を引き起こす可能性があるC-contiguous。
        
    n_jobs : int or None, optional (default=None)
        計算に使用するジョブの数これは計算によって機能します
        各n_initは並列に実行されます。
        `` None``は：obj： `joblib.parallel_backend`コンテキストの中でなければ1を意味します。
        `` -1``は全てのプロセッサを使うことを意味します。参照：term： `用語集<n_jobs>`
        
    algorithm : "auto"、 "full"、または "elkan"、デフォルト= "auto"
        使用するK平均アルゴリズム古典的なEMスタイルのアルゴリズムは "full"です。
        "elkan"バリエーションは三角形を使うことでもっと効率的です
        不等式ですが、現時点では疎データをサポートしていません。 「自動」が選択
        密データの場合は「elkan」、疎データの場合は「full」です。
        
    return_n_iter : bool, optional
        繰り返し回数を返すかどうか。
        
    Returns
    -------
    centroid : 形状（k、n_features）のfloat ndarray
        k平均の最後の反復で見つかった重心。
        
    label :形状付き整数ndarray（n_samples、）
        label [i]は、重心のコードまたはインデックスです。
        私は観測に最も近いです。
        
    inertia : float
        慣性基準の最終値（2乗距離の合計
        トレーニングセット内のすべての観測値に最も近い重心）

    best_n_iter : int
       最良の結果に対応する反復数。
        `return_n_iter`がTrueに設定されている場合にのみ返されます。
        
    """
    if n_init <= 0:
        raise ValueError("Invalid number of initializations."
                         " n_init=%d must be bigger than zero." % n_init)
    random_state = check_random_state(random_state)

    if max_iter <= 0:
        raise ValueError('Number of iterations should be a positive number,'
                         ' got %d instead' % max_iter)

    # avoid forcing order when copy_x=False
    order = "C" if copy_x else None
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32],
                    order=order, copy=copy_x)
    # verify that the number of samples given is larger than k
    if _num_samples(X) < n_clusters:
        raise ValueError("n_samples=%d should be >= n_clusters=%d" % (
            _num_samples(X), n_clusters))

    tol = _tolerance(X, tol)

    # If the distances are precomputed every job will create a matrix of shape
    # (n_clusters, n_samples). To stop KMeans from eating up memory we only
    # activate this if the created matrix is guaranteed to be under 100MB. 12
    # million entries consume a little under 100MB if they are of type double.
    if precompute_distances == 'auto':
        n_samples = X.shape[0]
        precompute_distances = (n_clusters * n_samples) < 12e6
    elif isinstance(precompute_distances, bool):
        pass
    else:
        raise ValueError("precompute_distances should be 'auto' or True/False"
                         ", but a value of %r was passed" %
                         precompute_distances)

    # Validate init array
    if hasattr(init, '__array__'):
        init = check_array(init, dtype=X.dtype.type, copy=True)
        _validate_center_shape(X, n_clusters, init)

        if n_init != 1:
            warnings.warn(
                'Explicit initial center position passed: '
                'performing only one init in k-means instead of n_init=%d'
                % n_init, RuntimeWarning, stacklevel=2)
            n_init = 1

    # subtract of mean of x for more accurate distance computations
    if not sp.issparse(X):
        X_mean = X.mean(axis=0)
        # The copy was already done above
        X -= X_mean

        if hasattr(init, '__array__'):
            init -= X_mean

    # precompute squared norms of data points
    x_squared_norms = row_norms(X, squared=True)

    best_labels, best_inertia, best_centers = None, None, None
    if n_clusters == 1:
        # elkan doesn't make sense for a single cluster, full will produce
        # the right result.
        algorithm = "full"
    if algorithm == "auto":
        algorithm = "full" if sp.issparse(X) else 'elkan'
    if algorithm == "full":
        kmeans_single = _kmeans_single_lloyd
    elif algorithm == "elkan":
        kmeans_single = _kmeans_single_elkan
    else:
        raise ValueError("Algorithm must be 'auto', 'full' or 'elkan', got"
                         " %s" % str(algorithm))
    if effective_n_jobs(n_jobs):
        # For a single thread, less memory is needed if we just store one set
        # of the best results (as opposed to one set per run per thread).
        for it in range(n_init):
            # run a k-means once
            labels, inertia, centers, n_iter_ = kmeans_single(
                X, sample_weight, n_clusters, max_iter=max_iter, init=init,
                verbose=verbose, precompute_distances=precompute_distances,
                tol=tol, x_squared_norms=x_squared_norms,
                random_state=random_state)
            # determine if these results are the best so far
            if best_inertia is None or inertia < best_inertia:
                best_labels = labels.copy()
                best_centers = centers.copy()
                best_inertia = inertia
                best_n_iter = n_iter_
    else:
        # parallelisation of k-means runs
        seeds = random_state.randint(np.iinfo(np.int32).max, size=n_init)
        results = Parallel(n_jobs=n_jobs, verbose=0)(
            delayed(kmeans_single)(X, sample_weight, n_clusters,
                                   max_iter=max_iter, init=init,
                                   verbose=verbose, tol=tol,
                                   precompute_distances=precompute_distances,
                                   x_squared_norms=x_squared_norms,
                                   # Change seed to ensure variety
                                   random_state=seed)
            for seed in seeds)
        # Get results with the lowest inertia
        labels, inertia, centers, n_iters = zip(*results)
        best = np.argmin(inertia)
        best_labels = labels[best]
        best_inertia = inertia[best]
        best_centers = centers[best]
        best_n_iter = n_iters[best]

    if not sp.issparse(X):
        if not copy_x:
            X += X_mean
        best_centers += X_mean

    distinct_clusters = len(set(best_labels))
    if distinct_clusters < n_clusters:
        warnings.warn("Number of distinct clusters ({}) found smaller than "
                      "n_clusters ({}). Possibly due to duplicate points "
                      "in X.".format(distinct_clusters, n_clusters),
                      ConvergenceWarning, stacklevel=2)

    if return_n_iter:
        return best_centers, best_labels, best_inertia, best_n_iter
    else:
        return best_centers, best_labels, best_inertia
```

In [ ]:
class ScratchKMeans():
    
    

In [1]:
# クラスタリング用人工データセット
from sklearn.datasets import make_blobs

In [2]:
# クラスタリング用の等方性ガウスブロブを生成する
X, _ = make_blobs(n_samples=100, n_features=2, centers=4, cluster_std=0.5, shuffle=True, random_state=0)

In [3]:
# 内容確認
X

array([[ 0.72086751,  3.71347124],
       [-1.89468423,  7.96898545],
       [ 1.35678894,  4.36462484],
       [ 1.05374379,  4.49286859],
       [ 1.59141542,  4.90497725],
       [ 0.78260667,  4.15263595],
       [-1.95751686,  3.87291474],
       [-0.77354537,  7.87923564],
       [ 0.12313498,  5.27917503],
       [-1.43284669,  7.71577043],
       [-0.92819001,  7.02698199],
       [-1.74836345,  7.06307447],
       [-1.26789718,  7.25141327],
       [-0.98661744,  7.74968685],
       [-0.81984047,  7.50994722],
       [ 2.99684287,  0.22378413],
       [ 1.46870582,  1.86947425],
       [-0.33533163,  3.390122  ],
       [-1.86407034,  2.93379754],
       [ 2.62496786,  0.28025075],
       [ 2.11114739,  3.57660449],
       [-1.8219901 ,  7.61654999],
       [-1.91186205,  3.18750686],
       [ 2.28809874,  0.12954182],
       [ 0.5285368 ,  4.49723858],
       [-1.57613028,  2.58614312],
       [-0.565433  ,  3.65813966],
       [ 0.802314  ,  4.38196181],
       [ 2.79939362,

In [4]:
# 内容確認
_

array([0, 3, 0, 0, 0, 0, 2, 3, 0, 3, 3, 3, 3, 3, 3, 1, 1, 2, 2, 1, 0, 3,
       2, 1, 0, 2, 2, 0, 1, 1, 1, 3, 1, 1, 2, 0, 3, 1, 3, 2, 0, 2, 3, 2,
       2, 3, 1, 2, 0, 0, 0, 1, 2, 2, 2, 3, 3, 1, 1, 3, 3, 1, 1, 0, 1, 3,
       2, 2, 1, 0, 3, 1, 0, 3, 0, 0, 2, 2, 1, 1, 1, 3, 2, 0, 1, 2, 1, 1,
       0, 0, 0, 2, 0, 2, 2, 3, 3, 2, 3, 0])